# Run thermal model

In [1]:
import finesse, pickle, glob, os, sys
import numpy as np
import matplotlib.pyplot as plt
# import finesse.analysis.actions as fac
from tqdm import tqdm
# from copy import deepcopy
# from finesse.knm import Map
# from finesse.ligo import maps
from finesse.ligo.factory import ALIGOFactory
# from finesse.ligo.actions import InitialLockLIGO
# from finesse.ligo.maps import get_test_mass_surface_profile_interpolated
from matplotlib.backends.backend_pdf import PdfPages

sys.path.append(str(finesse.ligo.git_path() / "LLO"))
sys.path.append(str(finesse.ligo.git_path() / "analysis" / "O4" / "GA_State_Optimization" / "src"))

from funcs import (model_output_params,
                   run_thermal_model,
                   generate_plots,
                   read_yaml_as_dict,
                   print_yamlparams_factory,
                   )

# from thermal_rom import make_ts_optics, add_thermal_detectors

use_real_data = False
initial_data_time = 1388643918 # to do: add ability to specify time and pull data.

In [2]:
suffix = '20250214'
override_RHeff = False
suff2 = ''

parentrepo = str(finesse.ligo.git_path())
repopath = str(finesse.ligo.git_path() / "analysis" / "O4" / "GA_State_Optimization")

yamls = glob.glob(f'{repopath}/data/run_{suffix}/cold_state_yaml_solutions/*.yaml')
yamls.sort()

figfolder = f'{repopath}/figures/figs_{suffix}'
os.makedirs(figfolder, exist_ok=True)

In [ ]:
factory = ALIGOFactory(f"{parentrepo}/LLO/yaml/llo_O4.yaml")
factory.update_parameters(f"{parentrepo}/LLO/yaml/llo_addRH.yaml")

# print params before update factory
print_yamlparams_factory(factory, read_yaml_as_dict(yamls[0]))

# yaml update factory
factory.update_parameters(yamls[0])

# print params after update factory
print()
print_yamlparams_factory(factory, read_yaml_as_dict(yamls[0]))

addl_parse = """
optbp OptQ_IMTXHRin ITMX.p2.i 0 fix_spot_size=false astigmatic=true accuracy=1e-09 direction=both
optbp OptQ_IMTXHRout ITMX.p1.o 0 fix_spot_size=false astigmatic=true accuracy=1e-09 direction=both
optbp OptQ_IMTYHRin ITMY.p2.i 0 fix_spot_size=false astigmatic=true accuracy=1e-09 direction=both
optbp OptQ_IMTYHRout ITMY.p1.o 0 fix_spot_size=false astigmatic=true accuracy=1e-09 direction=both
"""

In [ ]:
t_evol = 6000
llo = factory.make()

for i, yaml in enumerate(yamls[:1]):
    t, outs, models, locks, extra_outs, _ = run_thermal_model(
                                maxtems=8,
                                datapath=f'{repopath}/data/run_{suffix}/cold_state_yaml_solutions/thermal_evolution_{suffix}_{i}.pkl',
                                return_data_level=2,
                                update_yaml=yaml,
                                w0=1.015e-3,
                                z=6.0,
                                RH_efficiency={},
                                t_evol=t_evol,
                                tm_absorption={},
                                runsols=None,
                                model_output_params=model_output_params,
                                is_astig=True,
                                use_real_data=True,
                                repopath=parentrepo,
                                rec_therm_lenses=True,
                                override_RHeff=override_RHeff,
                                addl_parse=addl_parse,
                                new_bs=False,
                                )

In [5]:
fulldata = []

for i, yaml in enumerate(yamls[:1]):

    with open(f'{repopath}/data/run_{suffix}/cold_state_yaml_solutions/thermal_evolution_{suffix}_{i}.pkl', "rb") as file:
        data = pickle.load(file)

    fulldata.append({'t': data['t'], 'outs': data['outs'], 'varyParam': 'GA'})

# plot_thermal_evolution(fulldata, {'param': f'YAML{i}'},
#                        plotfileloc=f'{figfolder}/GA_state_powerup_{suffix}_{i}{suff2}.pdf',
#                        axislims=False)

# print(f'Fig Saved to:\n{figfolder}/GA_state_powerup_{suffix}_{i}{suff2}.pdf')

## Get real data

In [ ]:
use_real_data = True # needs to be true to plot data
# initial_data_time = 1400352800
initial_data_time = 1389224700

maxx = 6000

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

def get_data(chan_file: str = f'{parentrepo}/scripts/data.pkl'):
    '''
    To use real P_in data, in a terminal, first need to run
    >> python ligo-commissioning-modeling/scripts/data_munch.py L1_chans.yaml data.pkl
    for list of channels and times specified in L1_chans.yaml outputted to data.pkl
    can change the time interval in the L1_chans.yaml file
    '''
    with open(chan_file, 'rb') as f:
        data=pickle.load(f)
        return data

if use_real_data:
    print(f'Using real P_in data from IFO powerup at {initial_data_time}')
    d = get_data(f'{parentrepo}/scripts/data_{initial_data_time}.pkl')
    Power_data = d[initial_data_time]['data']['L1:IMC-IM4_TRANS_SUM_OUTPUT'].value
    Power_data_dt = d[initial_data_time]['data']['L1:IMC-IM4_TRANS_SUM_OUTPUT'].dt.value

# Sorting data for plotting
P_in = d[initial_data_time]['data']['L1:IMC-IM4_TRANS_SUM_OUTPUT'].value
t_data = np.linspace(0, len(P_in)*Power_data_dt, num = len(P_in))
irange = np.where(t_data < maxx)
t_data = t_data[irange]
P_in = P_in[irange] * 1
P_as = d[initial_data_time]['data']['L1:ASC-AS_C_SUM_OUTPUT'].value[irange] / 6400 # 6.4 cts/mW
P_refl = d[initial_data_time]['data']['L1:LSC-REFL_A_LF_OUTPUT'].value[irange] / 3370 # 3370 cts/W
P_pop = d[initial_data_time]['data']['L1:LSC-POP_A_LF_OUTPUT'].value[irange] * 0.92 # 0.92 W/cts
P_PRG = d[initial_data_time]['data']['L1:LSC-PRC_GAIN_MON'].value[irange] * 0.92 # 0.92 W/cts
Kappa_C = d[initial_data_time]['data']['L1:CAL-CS_TDEP_KAPPA_C_OUTPUT'].value[irange] # normalized to 1
RF18 = d[initial_data_time]['data']['L1:LSC-POPAIR_B_RF18_I_MON'].value[irange] * 0.0415 / P_in # 0.0415 W/cts per input W
RF90 = d[initial_data_time]['data']['L1:LSC-POPAIR_B_RF90_I_MON'].value[irange] * 0.018 / P_in # 0.018 W/cts per input W
# To do, could change RF18 and RF90 to Magnitude =  SQRT(I^2 + Q^2)

In [ ]:
# factory = ALIGOFactory(f"{parentrepo}/LLO/yaml/llo_O4.yaml")

outs = fulldata[0]['outs']
t = np.array(fulldata[0]['t'])

irange1 = np.where(np.array(t) < maxx)

finesse.init_plotting()

tollow, tolhigh = 0.95, 1.05
ylim = 1
plotloc = f'{figfolder}/thermalization_{initial_data_time}_{suffix}{suff2}.pdf'

print(plotloc)
generate_plots(plotloc ,initial_data_time, outs, irange1, t, 
               t_data, P_in, P_refl, P_pop, Kappa_C, P_PRG, 
               RF18, RF90, ylim, tollow, tolhigh, maxx, factory, savefig=True)

# RoC evolution

In [ ]:
# plot ITM lenses against time using extra_outs

t1 = t[1:]
# plotloc = f'{figfolder}/thermal_evol_of_TMs_{suffix}_{factory.params.RH_eff.ITMY:0.1f}.pdf'
plotloc = f'{figfolder}/thermal_evol_of_TM_RoC_{suffix}{suff2}.pdf'
print(plotloc)

for k, v in extra_outs.items():
    extra_outs[k] = np.array(v)

# beam front RoC evolution
beamfront_ITMX_Rc = []
beamfront_ITMY_Rc = []
for model in tqdm(models):
    model.beam_trace()
    tempsol = model.run("series(run_locks(exception_on_fail=False, max_iterations=500), noxaxis())")[1]
    beamfront_ITMX_Rc.append(-tempsol['OptQ_IMTXHRin'][0].Rc)
    beamfront_ITMY_Rc.append(-tempsol['OptQ_IMTXHRin'][1].Rc)

In [ ]:
# beam front RoC evolution
# beamfront_ITMX_Rc = []
# beamfront_ITMY_Rc = []
# for model in models:
#     propX = model.propagate_beam('PR2.p2.o', 'ITMX.p2.i', direction='x')
#     propY = model.propagate_beam('PR2.p2.o', 'ITMY.p2.i', direction='x')
#     beamfront_ITMX_Rc.append(propX.q(model.ITMX.p2.o).Rc)
#     beamfront_ITMY_Rc.append(propY.q(model.ITMY.p2.o).Rc)

with PdfPages(plotloc) as pdf:
    fig, axs = plt.subplots(3, 2, figsize=(16, 10))
    axs = axs.flatten()

    axs[0].plot(t1, 1e3/extra_outs['ITMXlens'], label='ITMX lens power')
    axs[0].plot(t1, 1e3/extra_outs['ITMYlens'], label='ITMY lens power')
    axs[0].set_title("ITM lenses")
    axs[0].set_xlabel('Time [s]')
    axs[0].set_ylabel('optical power mD')
    axs[0].set_xlim(1100, 6000)
    axs[0].legend()

    axs[1].plot(t1, 2e3/extra_outs['ITMX_Rc'], label='ITMX surface lens power')
    axs[1].plot(t1, 2e3/extra_outs['ITMY_Rc'], label='ITMY surface lens power')
    axs[1].set_title("ITM optical power due to surface deformation")
    axs[1].set_xlabel('Time [s]')
    axs[1].set_ylabel('optical power mD')
    axs[1].set_xlim(1100, 6000)
    axs[1].legend()

    axs[2].plot(t1, 2e3/extra_outs['ETMX_Rc'], label='ETMX surface lens power')
    axs[2].plot(t1, 2e3/extra_outs['ETMY_Rc'], label='ETMY surface lens power')
    axs[2].set_title("ETM optical power due to surface deformation")
    axs[2].set_xlabel('Time [s]')
    axs[2].set_ylabel('optical power mD')
    axs[2].set_xlim(1100, 6000)
    axs[2].legend()

    # axs[3].plot(t1, extra_outs['ITMX_Rc'], 'r', linewidth=2, label='ITMX surface RoC')
    # axs[3].plot(t1, extra_outs['ITMY_Rc'], 'b', linewidth=2, label='ITMY surface RoC')
    axs[3].plot(t1, beamfront_ITMX_Rc[1:], 'k:', label='Beamfront RoC @ ITMX')
    axs[3].plot(t1, beamfront_ITMY_Rc[1:], 'g:', label='Beamfront RoC @ ITMY')
    axs[3].set_title("Comparison of RoCs of beamfront and ITM surface")
    axs[3].set_xlabel('Time [s]')
    axs[3].set_ylabel('RoC [m]')
    axs[3].set_xlim(1100, 6000)
    axs[3].legend()

    axs[4].plot(t1, extra_outs['ITMX_Rc'], 'r', linewidth=2, label='ITMX surface RoC')
    axs[4].plot(t1, extra_outs['ITMY_Rc'], 'b', linewidth=2, label='ITMY surface RoC')
    # axs[4].plot(t1, beamfront_ITMX_Rc[1:], 'k:', label='Beamfront RoC @ ITMX')
    # axs[4].plot(t1, beamfront_ITMY_Rc[1:], 'g:', label='Beamfront RoC @ ITMY')
    axs[4].set_title("Comparison of RoCs of beamfront and ITM surface")
    axs[4].set_xlabel('Time [s]')
    axs[4].set_ylabel('RoC [m]')
    axs[4].set_xlim(1100, 6000)
    axs[4].legend()

    axs[5].plot(t1, extra_outs['ITMX_Rc'], 'r', linewidth=2, label='ITMX surface RoC')
    axs[5].plot(t1, extra_outs['ITMY_Rc'], 'b', linewidth=2, label='ITMY surface RoC')
    axs[5].plot(t1, beamfront_ITMX_Rc[1:], 'k:', label='Beamfront RoC @ ITMX')
    axs[5].plot(t1, beamfront_ITMY_Rc[1:], 'g:', label='Beamfront RoC @ ITMY')
    axs[5].set_title("Comparison of RoCs of beamfront and ITM surface")
    axs[5].set_xlabel('Time [s]')
    axs[5].set_ylabel('RoC [m]')
    axs[5].set_xlim(1100, 6000)
    axs[5].legend()

    plt.tight_layout()
    pdf.savefig()
    plt.show()

# Optimised q

In [ ]:
t, llo_drmi, sol1, llo, sols, runsols = run_thermal_model(
                                    maxtems=8,
                                    datapath=f'{repopath}/data/run_{suffix}/cold_state_yaml_solutions/thermal_evolution_{suffix}_{i}.pkl',
                                    return_data_level=1,
                                    update_yaml=yaml,
                                    w0=1.015e-3,
                                    z=6.0,
                                    t_evol=40,
                                    runsols=None,
                                    model_output_params=model_output_params,
                                    tm_absorption=tm_abs,
                                    RH_efficiency=RH_eff,
                                    is_astig=True,
                                    use_real_data=True,
                                    repopath=parentrepo,
                                    override_RHeff=override_RHeff,
                                    rec_therm_lenses=False,
                                    )